In [ ]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NaiveBaseClassifier/mushrooms.csv')

In [ ]:
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


Encode the Categorical data  into Numerical Data

In [ ]:
 le=LabelEncoder()
 #apply transformation each column 
 ds=df.apply(le.fit_transform)
#  ds

 


In [ ]:
ds.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [ ]:
#converting ds into nparray
data=ds.values
print(data.shape)
print(type(data))
print(data[:5,:])

data_y=data[:,0]
data_x=data[:,1:]

(8124, 23)
<class 'numpy.ndarray'>
[[1 5 2 4 1 6 1 0 1 4 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 9 1 0 1 0 0 4 0 2 2 2 7 7 0 2 1 4 3 2 1]
 [0 0 2 8 1 3 1 0 0 5 0 2 2 2 7 7 0 2 1 4 3 2 3]
 [1 5 3 8 1 6 1 0 1 5 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 3 0 5 1 1 0 4 1 3 2 2 7 7 0 2 1 0 3 0 1]]


Break the datda into train and test 

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(data_x,data_y,test_size=0.2)

In [ ]:
print(x_train.shape,y_test.shape)
print(x_test.shape,y_test.shape)

(6499, 22) (1625,)
(1625, 22) (1625,)


In [ ]:
np.unique(y_train)

array([0, 1])

Building Our Classifier

In [ ]:
a=np.array([0,0,0,1,1,1,0,1])
np.sum(a==1)

4

In [ ]:
 def prior_prob(y_train,label):
  total_example=y_train.shape[0]
  class_example=np.sum(y_train==label)
  return (class_example)/float(total_example)



In [ ]:
#compute prior probability
y=np.array([0,5,5,1,1,1,1,0,0,0])
prior_prob(y,5)


0.2

In [ ]:
def cond_prob(x_train,y_train,feature_col,feature_val,label):
  x_filtered = x_train[y_train == label]

  numerator=np.sum(x_filtered[:,feature_col]==feature_val)
  denominator=np.sum(y_train==label)
  return numerator/float(denominator)
  #p(x=green|y=2) =2/3

Next Step:Compute Posterier prob for each test example make predictions


In [ ]:
def predict (x_train,y_train,xtest ):
  """Xtest is the single testing point ,n features"""

  classes=np.unique(y_train)
  n_features=x_train.shape[1]
  post_probs=[]#list of prob for all classes and given a single testing point 
  #Compute Posterior for each class 
  for label in classes:
    #post_C=likelihood*prior
    likehood =1.0
    for f in range (n_features):
      cond=cond_prob(x_train,y_train,f,xtest[f],label)
      likehood*=cond
    prior=prior_prob(y_train,label)
    post=likehood*prior
    post_probs.append(post)    
  pred=np.argmax(post_probs)
  return pred

In [ ]:
output=predict(x_train,y_train,x_test[1])
print(output)
print(y_test[1])

1
1


In [ ]:
def score (x_train,y_train,x_test,_test):
  pred=[]
  for i in range (x_test.shape[0]):
    pred_label=predict(x_train,y_train,x_test[i])
    pred.append(pred_label)#correlation
  pred=np.array(pred)
  accuracy=np.sum(pred==y_test)/y_test.shape[0]
  return accuracy


# def score(x_train, y_train, x_test, y_test):
#     pred = []  # Initialize the 'pred' variable
#     for i in range(x_test.shape[0]):
#         pred_label = predict(x_train, y_train, x_test[i])
#         pred.append(pred_label)
#     pred = np.array(pred)
#     accuracy = np.sum(pred == y_test) / y_test.shape[0]
#     return accuracy


In [ ]:
print(score(x_train,y_train,x_test,y_test))

0.9969230769230769
